# TODO  
## MySQLにも保存

# importなど

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
from dateutil.parser import parse

import stock

%matplotlib inline

## 結合済みファイルの読み込み

In [297]:
kessan_table = pd.read_csv('/Users/Really/Stockyard/_csv/kessan_table.csv')

In [315]:
kessan_table

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目（売上高欄に掲載）,売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開又は更新日
0,1301,極洋,日本基準,個別,2013年3月期,第2四半期,2012-04-01,2012-09-30,売上高,7.654600e+10,...,7.900000e+07,0.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-10-21
1,1301,極洋,日本基準,個別,2013年3月期,通期,2012-04-01,2013-03-31,売上高,1.606100e+11,...,8.060000e+08,7.68,NaN,1.481800e+10,7.049200e+10,141.08,NaN,NaN,NaN,2014-05-09
2,1301,極洋,日本基準,個別,2014年3月期,第2四半期,2013-04-01,2013-09-30,売上高,8.816100e+10,...,1.834000e+09,17.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-10-27
3,1301,極洋,日本基準,個別,2014年3月期,通期,2013-04-01,2014-03-31,売上高,1.920260e+11,...,1.806000e+09,17.20,16.77,1.616400e+10,7.038100e+10,153.90,NaN,NaN,NaN,2015-05-08
4,1301,極洋,日本基準,個別,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,9.947600e+10,...,2.504000e+09,23.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-26
5,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.120860e+11,...,2.139000e+09,20.37,18.78,1.769500e+10,7.351900e+10,168.48,NaN,NaN,NaN,2015-05-08
6,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.120860e+11,...,2.139000e+09,20.37,18.78,1.769500e+10,7.351900e+10,168.48,NaN,NaN,NaN,2016-05-09
7,1301,極洋,日本基準,個別,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2.163500e+11,...,1.259000e+09,119.87,110.51,1.781700e+10,7.814000e+10,1696.42,NaN,NaN,NaN,2017-05-11
8,1301,極洋,日本基準,個別,2017年3月期,通期,2016-04-01,2017-03-31,売上高,2.280830e+11,...,2.194000e+09,208.94,192.95,2.011700e+10,8.130600e+10,1915.42,NaN,NaN,NaN,2017-05-11
9,1301,極洋,日本基準,連結,2013年3月期,第1四半期,2012-04-01,2012-06-30,売上高,4.319100e+10,...,3.350000e+08,3.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-05


csv では読み込みのたびに型を設定しなおさなければならず面倒。  
整形後に保存した csv かつ整形済みのデータが実行環境内に存在する場合、その型の配列を辞書にして適用することも可能。

In [308]:
datatype = dict(kessan_table_15.dtypes)
datatype

{'一株当り純利益': dtype('float64'),
 '一株当り純資産': dtype('float64'),
 '企業名': dtype('O'),
 '会計基準': dtype('O'),
 '名寄前勘定科目（売上高欄に掲載）': dtype('O'),
 '営業キャッシュフロー': dtype('float64'),
 '営業利益': dtype('float64'),
 '売上高': dtype('float64'),
 '希薄化後一株当り純利益': dtype('float64'),
 '情報公開又は更新日': dtype('<M8[ns]'),
 '投資キャッシュフロー': dtype('float64'),
 '期末': dtype('<M8[ns]'),
 '期首': dtype('<M8[ns]'),
 '決算期': dtype('O'),
 '決算期間': dtype('O'),
 '純利益': dtype('float64'),
 '純資産又は株主資本': dtype('float64'),
 '経常利益': dtype('float64'),
 '総資産': dtype('float64'),
 '証券コード': dtype('int64'),
 '財務キャッシュフロー': dtype('float64'),
 '連結個別': dtype('O')}

In [312]:
kessan_table = kessan_table.astype(datatype)

In [316]:
kessan_table.dtypes

pandas.core.series.Series

# ファイルの読み込み、整形、保存

## xlsのまま読み込む  
この方がデータの型が維持されるので手間が少なく済む  
csvよりも読み込みに少し時間がかかる

In [2]:
# 決算プロの決算短信xlsを読み込む
kessan_table_new = pd.read_excel('/Users/Really/Stockyard/_dl_data/20171112f.xls') # http://ke.kabupro.jp/doc/down40.htm

## 内容の確認

In [3]:
kessan_table_new

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目（売上高欄に掲載）,売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開又は更新日
0,1301,極洋,日本基準,連結,2018年3月期,第2四半期,2017-04-01,2017-09-30,売上高,1.204580e+11,...,1.633000e+09,155.50,144.15,2.669200e+10,1.198060e+11,NaN,NaN,NaN,NaN,2017-11-06
1,1301,極洋,日本基準,連結,2018年3月期,第1四半期,2017-04-01,2017-06-30,売上高,5.684400e+10,...,7.540000e+08,71.80,66.56,2.556000e+10,1.074220e+11,NaN,NaN,NaN,NaN,2017-08-04
2,1301,極洋,日本基準,連結,2017年3月期,通期,2016-04-01,2017-03-31,売上高,2.365610e+11,...,2.422000e+09,230.66,213.01,2.539100e+10,9.739100e+10,2378.09,6.010000e+08,-1.998000e+09,1.050000e+08,2017-05-11
3,1301,極洋,日本基準,連結,2017年3月期,第3四半期,2016-04-01,2016-12-31,売上高,1.799750e+11,...,2.449000e+09,233.26,215.13,2.577900e+10,1.171680e+11,NaN,NaN,NaN,NaN,2017-02-10
4,1301,極洋,日本基準,連結,2017年3月期,第2四半期,2016-04-01,2016-09-30,売上高,1.095700e+11,...,1.186000e+09,112.99,104.16,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-06
5,1301,極洋,日本基準,連結,2017年3月期,第1四半期,2016-04-01,2016-06-30,売上高,5.220600e+10,...,5.510000e+08,52.47,48.37,NaN,NaN,NaN,NaN,NaN,NaN,2017-08-04
6,1301,極洋,日本基準,連結,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2.266260e+11,...,1.799000e+09,171.33,157.94,2.306500e+10,9.460800e+10,2149.67,2.689000e+09,-5.114000e+09,2.482000e+09,2017-05-11
7,1301,極洋,日本基準,連結,2016年3月期,第3四半期,2015-04-01,2015-12-31,売上高,1.788900e+11,...,1.133000e+09,107.94,99.51,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-10
8,1301,極洋,日本基準,連結,2016年3月期,第2四半期,2015-04-01,2015-09-30,売上高,1.084990e+11,...,9.110000e+08,86.74,79.97,NaN,NaN,NaN,NaN,NaN,NaN,2016-11-04
9,1301,極洋,日本基準,連結,2016年3月期,第1四半期,2015-04-01,2015-06-30,売上高,5.016800e+10,...,3.160000e+08,3.02,2.78,NaN,NaN,NaN,NaN,NaN,NaN,2016-08-05


In [5]:
kessan_table_new.columns

Index(['証券コード', '企業名', '会計基準', '連結個別', '決算期', '決算期間', '期首', '期末',
       '名寄前勘定科目（売上高欄に掲載）', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益',
       '希薄化後一株当り純利益', '純資産又は株主資本', '総資産', '一株当り純資産', '営業キャッシュフロー',
       '投資キャッシュフロー', '財務キャッシュフロー', '情報公開又は更新日'],
      dtype='object')

In [6]:
kessan_table_new[['証券コード', '企業名', '会計基準', '連結個別', '決算期', '決算期間', '期首', '期末',
       '名寄前勘定科目（売上高欄に掲載）', '情報公開又は更新日']]

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目（売上高欄に掲載）,情報公開又は更新日
0,1301,極洋,日本基準,連結,2018年3月期,第2四半期,2017-04-01,2017-09-30,売上高,2017-11-06
1,1301,極洋,日本基準,連結,2018年3月期,第1四半期,2017-04-01,2017-06-30,売上高,2017-08-04
2,1301,極洋,日本基準,連結,2017年3月期,通期,2016-04-01,2017-03-31,売上高,2017-05-11
3,1301,極洋,日本基準,連結,2017年3月期,第3四半期,2016-04-01,2016-12-31,売上高,2017-02-10
4,1301,極洋,日本基準,連結,2017年3月期,第2四半期,2016-04-01,2016-09-30,売上高,2017-11-06
5,1301,極洋,日本基準,連結,2017年3月期,第1四半期,2016-04-01,2016-06-30,売上高,2017-08-04
6,1301,極洋,日本基準,連結,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2017-05-11
7,1301,極洋,日本基準,連結,2016年3月期,第3四半期,2015-04-01,2015-12-31,売上高,2017-02-10
8,1301,極洋,日本基準,連結,2016年3月期,第2四半期,2015-04-01,2015-09-30,売上高,2016-11-04
9,1301,極洋,日本基準,連結,2016年3月期,第1四半期,2015-04-01,2015-06-30,売上高,2016-08-05


In [9]:
kessan_table_new[['証券コード', '企業名', '期末', '売上高', '営業利益', '経常利益', '純利益', '情報公開又は更新日']]

,証券コード,企業名,期末,売上高,営業利益,経常利益,純利益,情報公開又は更新日
0,1301,極洋,2017-09-30,1.204580e+11,2.246000e+09,2.396000e+09,1.633000e+09,2017-11-06
1,1301,極洋,2017-06-30,5.684400e+10,9.790000e+08,1.103000e+09,7.540000e+08,2017-08-04
2,1301,極洋,2017-03-31,2.365610e+11,3.723000e+09,3.709000e+09,2.422000e+09,2017-05-11
3,1301,極洋,2016-12-31,1.799750e+11,2.872000e+09,2.827000e+09,2.449000e+09,2017-02-10
4,1301,極洋,2016-09-30,1.095700e+11,1.171000e+09,1.004000e+09,1.186000e+09,2017-11-06
5,1301,極洋,2016-06-30,5.220600e+10,4.670000e+08,3.800000e+08,5.510000e+08,2017-08-04
6,1301,極洋,2016-03-31,2.266260e+11,2.433000e+09,2.814000e+09,1.799000e+09,2017-05-11
7,1301,極洋,2015-12-31,1.788900e+11,2.467000e+09,2.688000e+09,1.133000e+09,2017-02-10
8,1301,極洋,2015-09-30,1.084990e+11,1.126000e+09,1.407000e+09,9.110000e+08,2016-11-04
9,1301,極洋,2015-06-30,5.016800e+10,2.730000e+08,4.920000e+08,3.160000e+08,2016-08-05


In [8]:
kessan_table_new[['証券コード', '企業名', '期末', '一株当り純利益', '希薄化後一株当り純利益', '純資産又は株主資本',
       '総資産', '一株当り純資産', '営業キャッシュフロー', '投資キャッシュフロー', '財務キャッシュフロー',
       '情報公開又は更新日']]

,証券コード,企業名,期末,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開又は更新日
0,1301,極洋,2017-09-30,155.50,144.15,2.669200e+10,1.198060e+11,NaN,NaN,NaN,NaN,2017-11-06
1,1301,極洋,2017-06-30,71.80,66.56,2.556000e+10,1.074220e+11,NaN,NaN,NaN,NaN,2017-08-04
2,1301,極洋,2017-03-31,230.66,213.01,2.539100e+10,9.739100e+10,2378.09,6.010000e+08,-1.998000e+09,1.050000e+08,2017-05-11
3,1301,極洋,2016-12-31,233.26,215.13,2.577900e+10,1.171680e+11,NaN,NaN,NaN,NaN,2017-02-10
4,1301,極洋,2016-09-30,112.99,104.16,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-06
5,1301,極洋,2016-06-30,52.47,48.37,NaN,NaN,NaN,NaN,NaN,NaN,2017-08-04
6,1301,極洋,2016-03-31,171.33,157.94,2.306500e+10,9.460800e+10,2149.67,2.689000e+09,-5.114000e+09,2.482000e+09,2017-05-11
7,1301,極洋,2015-12-31,107.94,99.51,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-10
8,1301,極洋,2015-09-30,86.74,79.97,NaN,NaN,NaN,NaN,NaN,NaN,2016-11-04
9,1301,極洋,2015-06-30,3.02,2.78,NaN,NaN,NaN,NaN,NaN,NaN,2016-08-05


In [4]:
kessan_table_new.dtypes

証券コード                        int64
企業名                         object
会計基準                        object
連結個別                        object
決算期                         object
決算期間                        object
期首                  datetime64[ns]
期末                  datetime64[ns]
名寄前勘定科目（売上高欄に掲載）            object
売上高                        float64
営業利益                       float64
経常利益                       float64
純利益                        float64
一株当り純利益                    float64
希薄化後一株当り純利益                float64
純資産又は株主資本                  float64
総資産                        float64
一株当り純資産                    float64
営業キャッシュフロー                 float64
投資キャッシュフロー                 float64
財務キャッシュフロー                 float64
情報公開又は更新日           datetime64[ns]
dtype: object

In [10]:
kessan_table_new.duplicated().any()

False

In [11]:
kessan_table_new.isnull().any()

証券コード               False
企業名                 False
会計基準                False
連結個別                False
決算期                 False
決算期間                False
期首                  False
期末                  False
名寄前勘定科目（売上高欄に掲載）     True
売上高                  True
営業利益                 True
経常利益                 True
純利益                  True
一株当り純利益              True
希薄化後一株当り純利益          True
純資産又は株主資本            True
総資産                  True
一株当り純資産              True
営業キャッシュフロー           True
投資キャッシュフロー           True
財務キャッシュフロー           True
情報公開又は更新日           False
dtype: bool

In [12]:
kessan_table_new['期末'].min()

Timestamp('2014-03-31 00:00:00')

In [13]:
kessan_table_new['情報公開又は更新日'].min()

Timestamp('2014-04-25 00:00:00')

In [20]:
kessan_table_new.groupby(['連結個別', '会計基準']).count()

証券コード    企業名    決算期   決算期間     期首     期末  名寄前勘定科目（売上高欄に掲載）    売上高  \
連結個別 会計基準                                                                      
個別   日本基準  16195  16195  16195  16195  16195  16195             16170  16170   
連結   日本基準  40993  40993  40993  40993  40993  40993             40978  40978   
     米国基準   1781   1781   1781   1781   1781   1781              1742   1742   

            営業利益   経常利益    純利益  一株当り純利益  希薄化後一株当り純利益  純資産又は株主資本    総資産  \
連結個別 会計基準                                                                
個別   日本基準  15108  16193  16195    16174         3786      11331  11331   
連結   日本基準  39639  40987  40983    40976        10909      17445  17445   
     米国基準   1665   1761   1780     1778         1058        789    789   

           一株当り純資産  営業キャッシュフロー  投資キャッシュフロー  財務キャッシュフロー  情報公開又は更新日  
連結個別 会計基準                                                          
個別   日本基準    10032        1369        1367        1344      16195  
連結   日本基準    11378        7904        7903        7892      40993  
     米国基準      477         341         341         341       1781

In [36]:
kessan_table_new[kessan_table_new['連結個別'] == '個別']

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目（売上高欄に掲載）,売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開又は更新日
14,1301,極洋,日本基準,個別,2017年3月期,通期,2016-04-01,2017-03-31,売上高,2.280830e+11,...,2.194000e+09,208.94,192.95,2.011700e+10,8.130600e+10,1915.42,NaN,NaN,NaN,2017-05-11
15,1301,極洋,日本基準,個別,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2.163500e+11,...,1.259000e+09,119.87,110.51,1.781700e+10,7.814000e+10,1696.42,NaN,NaN,NaN,2017-05-11
16,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.120860e+11,...,2.139000e+09,20.37,18.78,1.769500e+10,7.351900e+10,168.48,NaN,NaN,NaN,2016-05-09
17,1301,極洋,日本基準,個別,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,9.947600e+10,...,2.504000e+09,23.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-26
32,1332,日本水産,日本基準,個別,2017年3月期,通期,2016-04-01,2017-03-31,売上高,3.613440e+11,...,7.924000e+09,26.76,NaN,8.170900e+10,3.016090e+11,262.21,NaN,NaN,NaN,2017-05-15
33,1332,日本水産,日本基準,個別,2016年3月期,通期,2015-04-01,2016-03-31,売上高,3.576560e+11,...,7.119000e+09,25.76,NaN,6.106100e+10,2.932350e+11,220.91,NaN,NaN,NaN,2017-05-15
34,1332,日本水産,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,3.506830e+11,...,4.155000e+09,15.03,NaN,5.686700e+10,2.983910e+11,205.73,NaN,NaN,NaN,2017-01-20
49,1333,マルハ,日本基準,個別,2017年3月期,通期,2016-04-01,2017-03-31,売上高,4.241800e+11,...,1.045900e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-05-08
50,1333,マルハ,日本基準,個別,2016年3月期,通期,2015-04-01,2016-03-31,売上高,4.211710e+11,...,3.837000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-05-08
51,1333,マルハ,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,4.146260e+11,...,-9.946000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-05-09


In [37]:
kessan_table_new[kessan_table_new['連結個別'] == '連結']

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目（売上高欄に掲載）,売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開又は更新日
0,1301,極洋,日本基準,連結,2018年3月期,第2四半期,2017-04-01,2017-09-30,売上高,1.204580e+11,...,1.633000e+09,155.50,144.15,2.669200e+10,1.198060e+11,NaN,NaN,NaN,NaN,2017-11-06
1,1301,極洋,日本基準,連結,2018年3月期,第1四半期,2017-04-01,2017-06-30,売上高,5.684400e+10,...,7.540000e+08,71.80,66.56,2.556000e+10,1.074220e+11,NaN,NaN,NaN,NaN,2017-08-04
2,1301,極洋,日本基準,連結,2017年3月期,通期,2016-04-01,2017-03-31,売上高,2.365610e+11,...,2.422000e+09,230.66,213.01,2.539100e+10,9.739100e+10,2378.09,6.010000e+08,-1.998000e+09,1.050000e+08,2017-05-11
3,1301,極洋,日本基準,連結,2017年3月期,第3四半期,2016-04-01,2016-12-31,売上高,1.799750e+11,...,2.449000e+09,233.26,215.13,2.577900e+10,1.171680e+11,NaN,NaN,NaN,NaN,2017-02-10
4,1301,極洋,日本基準,連結,2017年3月期,第2四半期,2016-04-01,2016-09-30,売上高,1.095700e+11,...,1.186000e+09,112.99,104.16,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-06
5,1301,極洋,日本基準,連結,2017年3月期,第1四半期,2016-04-01,2016-06-30,売上高,5.220600e+10,...,5.510000e+08,52.47,48.37,NaN,NaN,NaN,NaN,NaN,NaN,2017-08-04
6,1301,極洋,日本基準,連結,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2.266260e+11,...,1.799000e+09,171.33,157.94,2.306500e+10,9.460800e+10,2149.67,2.689000e+09,-5.114000e+09,2.482000e+09,2017-05-11
7,1301,極洋,日本基準,連結,2016年3月期,第3四半期,2015-04-01,2015-12-31,売上高,1.788900e+11,...,1.133000e+09,107.94,99.51,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-10
8,1301,極洋,日本基準,連結,2016年3月期,第2四半期,2015-04-01,2015-09-30,売上高,1.084990e+11,...,9.110000e+08,86.74,79.97,NaN,NaN,NaN,NaN,NaN,NaN,2016-11-04
9,1301,極洋,日本基準,連結,2016年3月期,第1四半期,2015-04-01,2015-06-30,売上高,5.016800e+10,...,3.160000e+08,3.02,2.78,NaN,NaN,NaN,NaN,NaN,NaN,2016-08-05


In [38]:
kessan_table_new.describe()

,証券コード,売上高,営業利益,経常利益,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー
count,58969.000000,5.889000e+04,5.641200e+04,5.894100e+04,5.895800e+04,58928.000000,15753.000000,2.956500e+04,2.956500e+04,21887.000000,9.614000e+03,9.611000e+03,9.577000e+03
mean,5832.302922,1.237426e+11,7.602876e+09,9.447355e+09,6.089828e+09,49.314229,66.515108,1.241750e+11,6.906529e+11,1119.091200,2.651936e+10,-2.702570e+09,-3.231654e+09
std,2428.888901,5.830953e+11,4.628122e+10,5.723010e+10,4.223220e+10,108.580784,95.355665,6.448516e+11,8.132317e+12,1339.878413,2.537095e+11,3.311693e+11,6.130280e+10
min,1301.000000,-3.720000e+08,-5.762770e+11,-5.970170e+11,-1.092006e+12,-4468.650000,-185.370000,-7.303370e+11,1.060000e+08,-3308.020000,-2.922978e+12,-4.213597e+12,-2.130644e+12
25%,3676.000000,4.971000e+09,1.350000e+08,1.580000e+08,8.600000e+07,6.270000,15.840000,4.652000e+09,9.882000e+09,346.305000,3.450000e+08,-3.843000e+09,-1.866000e+09
50%,6113.000000,1.663200e+10,6.890000e+08,8.040000e+08,5.040000e+08,27.810000,38.080000,1.556300e+10,3.091600e+10,750.370000,1.568500e+09,-7.510000e+08,-3.430000e+08
75%,7912.000000,5.783225e+10,2.974000e+09,3.609000e+09,2.354000e+09,69.752500,81.680000,5.644600e+10,1.115610e+11,1419.360000,6.863750e+09,-1.090000e+08,9.700000e+07
max,9997.000000,2.840312e+13,2.853971e+12,2.983381e+12,2.745949e+12,2903.850000,2838.970000,1.940619e+13,3.042216e+14,21438.660000,9.752429e+12,1.552178e+13,2.380746e+12


## 整形、保存

In [25]:
# ソート
kessan_table_new = kessan_table_new.sort_values(['証券コード', '連結個別', '期末', '情報公開又は更新日']).reset_index(drop=True)

In [26]:
# 保存
kessan_table_new.to_csv('/Users/Really/Stockyard/_csv/kessan_table_20171112.csv', index=False)

In [27]:
# 保存したファイルの確認
df = pd.read_csv('/Users/Really/Stockyard/_csv/kessan_table_20171112.csv')
df

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目（売上高欄に掲載）,売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開又は更新日
0,1301,極洋,日本基準,個別,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,9.947600e+10,...,2.504000e+09,23.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-26
1,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.120860e+11,...,2.139000e+09,20.37,18.78,1.769500e+10,7.351900e+10,168.48,NaN,NaN,NaN,2016-05-09
2,1301,極洋,日本基準,個別,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2.163500e+11,...,1.259000e+09,119.87,110.51,1.781700e+10,7.814000e+10,1696.42,NaN,NaN,NaN,2017-05-11
3,1301,極洋,日本基準,個別,2017年3月期,通期,2016-04-01,2017-03-31,売上高,2.280830e+11,...,2.194000e+09,208.94,192.95,2.011700e+10,8.130600e+10,1915.42,NaN,NaN,NaN,2017-05-11
4,1301,極洋,日本基準,連結,2015年3月期,第1四半期,2014-04-01,2014-06-30,売上高,4.867500e+10,...,2.230000e+08,2.13,1.96,NaN,NaN,NaN,NaN,NaN,NaN,2015-08-03
5,1301,極洋,日本基準,連結,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,1.024760e+11,...,2.265000e+09,21.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-26
6,1301,極洋,日本基準,連結,2015年3月期,第3四半期,2014-04-01,2014-12-31,売上高,1.701400e+11,...,2.806000e+09,26.72,24.63,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-05
7,1301,極洋,日本基準,連結,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.183500e+11,...,2.433000e+09,23.17,21.36,2.306900e+10,8.893700e+10,215.65,-2.340000e+09,-7.620000e+08,3.698000e+09,2016-05-09
8,1301,極洋,日本基準,連結,2016年3月期,第1四半期,2015-04-01,2015-06-30,売上高,5.016800e+10,...,3.160000e+08,3.02,2.78,NaN,NaN,NaN,NaN,NaN,NaN,2016-08-05
9,1301,極洋,日本基準,連結,2016年3月期,第2四半期,2015-04-01,2015-09-30,売上高,1.084990e+11,...,9.110000e+08,86.74,79.97,NaN,NaN,NaN,NaN,NaN,NaN,2016-11-04


# 別アプリでcsvに変換してから読み込む場合

In [295]:
# 決算プロの決算短信csv(エクセルファイルより書き出し)を読み込む http://ke.kabupro.jp/doc/down40.htm
kessan_table_new_csv = pd.read_csv('/Users/Really/Stockyard/_dl_data/kabupro_20171112f.csv') 

## 内容の確認

In [58]:
kessan_table_new_csv

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目（売上高欄に掲載）,売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開又は更新日
0,1301,極洋,日本基準,連結,2015年3月期,第1四半期,2014-04-01,2014-06-30,売上高,4.867500e+10,...,2.230000e+08,2.13,1.96,NaN,NaN,NaN,NaN,NaN,NaN,2015-08-03
1,1301,極洋,日本基準,連結,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,1.024760e+11,...,2.265000e+09,21.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-26
2,1301,極洋,日本基準,個別,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,9.947600e+10,...,2.504000e+09,23.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-26
3,1301,極洋,日本基準,連結,2015年3月期,第3四半期,2014-04-01,2014-12-31,売上高,1.701400e+11,...,2.806000e+09,26.72,24.63,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-05
4,1301,極洋,日本基準,連結,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.183500e+11,...,2.433000e+09,23.17,21.36,2.306900e+10,8.893700e+10,215.65,-2.340000e+09,-7.620000e+08,3.698000e+09,2016-05-09
5,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.120860e+11,...,2.139000e+09,20.37,18.78,1.769500e+10,7.351900e+10,168.48,NaN,NaN,NaN,2016-05-09
6,1301,極洋,日本基準,連結,2016年3月期,第1四半期,2015-04-01,2015-06-30,売上高,5.016800e+10,...,3.160000e+08,3.02,2.78,NaN,NaN,NaN,NaN,NaN,NaN,2016-08-05
7,1301,極洋,日本基準,連結,2016年3月期,第2四半期,2015-04-01,2015-09-30,売上高,1.084990e+11,...,9.110000e+08,86.74,79.97,NaN,NaN,NaN,NaN,NaN,NaN,2016-11-04
8,1301,極洋,日本基準,連結,2016年3月期,第3四半期,2015-04-01,2015-12-31,売上高,1.788900e+11,...,1.133000e+09,107.94,99.51,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-10
9,1301,極洋,日本基準,連結,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2.266260e+11,...,1.799000e+09,171.33,157.94,2.306500e+10,9.460800e+10,2149.67,2.689000e+09,-5.114000e+09,2.482000e+09,2017-05-11


csv からの読み込みではほとんどの列の型が object になる (読み込み時にオプションで指定することも可能らしいが、列が多いと面倒)

In [296]:
kessan_table_new_csv.dtypes

証券コード                int64
企業名                 object
会計基準                object
連結個別                object
決算期                 object
決算期間                object
期首                  object
期末                  object
名寄前勘定科目（売上高欄に掲載）    object
売上高                 object
営業利益                object
経常利益                object
純利益                 object
一株当り純利益             object
希薄化後一株当り純利益         object
純資産又は株主資本           object
総資産                 object
一株当り純資産             object
営業キャッシュフロー          object
投資キャッシュフロー          object
財務キャッシュフロー          object
情報公開又は更新日           object
dtype: object

In [24]:
kessan_table_new_csv.duplicated().any()

False

In [25]:
kessan_table_new_csv.isnull().any()

証券コード               False
企業名                 False
会計基準                False
連結個別                False
決算期                 False
決算期間                False
期首                  False
期末                  False
名寄前勘定科目（売上高欄に掲載）     True
売上高                  True
営業利益                 True
経常利益                 True
純利益                  True
一株当り純利益              True
希薄化後一株当り純利益          True
純資産又は株主資本            True
総資産                  True
一株当り純資産              True
営業キャッシュフロー           True
投資キャッシュフロー           True
財務キャッシュフロー           True
情報公開又は更新日           False
dtype: bool

In [62]:
kessan_table_new_csv['期末'].min()

Timestamp('2014-03-31 00:00:00')

In [63]:
kessan_table_new_csv['情報公開又は更新日'].min()

Timestamp('2014-04-25 00:00:00')

In [29]:
kessan_table_new_csv[kessan_table_new_csv['連結個別'] == '個別']

,level_0,index,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開又は更新日
2,2,17,1301,極洋,日本基準,個別,2015年3月期,第2四半期,2014-04-01,2014-09-30,...,2.504000e+09,23.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-26
5,5,16,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,...,2.139000e+09,20.37,18.78,1.769500e+10,7.351900e+10,168.48,NaN,NaN,NaN,2016-05-09
10,10,15,1301,極洋,日本基準,個別,2016年3月期,通期,2015-04-01,2016-03-31,...,1.259000e+09,119.87,110.51,1.781700e+10,7.814000e+10,1696.42,NaN,NaN,NaN,2017-05-11
15,15,14,1301,極洋,日本基準,個別,2017年3月期,通期,2016-04-01,2017-03-31,...,2.194000e+09,208.94,192.95,2.011700e+10,8.130600e+10,1915.42,NaN,NaN,NaN,2017-05-11
22,22,34,1332,日本水産,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,...,4.155000e+09,15.03,NaN,5.686700e+10,2.983910e+11,205.73,NaN,NaN,NaN,2017-01-20
27,27,33,1332,日本水産,日本基準,個別,2016年3月期,通期,2015-04-01,2016-03-31,...,7.119000e+09,25.76,NaN,6.106100e+10,2.932350e+11,220.91,NaN,NaN,NaN,2017-05-15
32,32,32,1332,日本水産,日本基準,個別,2017年3月期,通期,2016-04-01,2017-03-31,...,7.924000e+09,26.76,NaN,8.170900e+10,3.016090e+11,262.21,NaN,NaN,NaN,2017-05-15
39,39,51,1333,マルハ,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,...,-9.946000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-05-09
44,44,50,1333,マルハ,日本基準,個別,2016年3月期,通期,2015-04-01,2016-03-31,...,3.837000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-05-08
49,49,49,1333,マルハ,日本基準,個別,2017年3月期,通期,2016-04-01,2017-03-31,...,1.045900e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-05-08


In [15]:
kessan_table_new_csv[kessan_table_new_csv['連結個別'] == '連結']

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目（売上高欄に掲載）,売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開又は更新日
0,1301,極洋,日本基準,連結,2015年3月期,第1四半期,2014-04-01,2014-06-30,売上高,4.867500e+10,...,2.230000e+08,2.13,1.96,NaN,NaN,NaN,NaN,NaN,NaN,2015-08-03
1,1301,極洋,日本基準,連結,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,1.024760e+11,...,2.265000e+09,21.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-26
3,1301,極洋,日本基準,連結,2015年3月期,第3四半期,2014-04-01,2014-12-31,売上高,1.701400e+11,...,2.806000e+09,26.72,24.63,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-05
4,1301,極洋,日本基準,連結,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.183500e+11,...,2.433000e+09,23.17,21.36,2.306900e+10,8.893700e+10,215.65,-2.340000e+09,-7.620000e+08,3.698000e+09,2016-05-09
6,1301,極洋,日本基準,連結,2016年3月期,第1四半期,2015-04-01,2015-06-30,売上高,5.016800e+10,...,3.160000e+08,3.02,2.78,NaN,NaN,NaN,NaN,NaN,NaN,2016-08-05
7,1301,極洋,日本基準,連結,2016年3月期,第2四半期,2015-04-01,2015-09-30,売上高,1.084990e+11,...,9.110000e+08,86.74,79.97,NaN,NaN,NaN,NaN,NaN,NaN,2016-11-04
8,1301,極洋,日本基準,連結,2016年3月期,第3四半期,2015-04-01,2015-12-31,売上高,1.788900e+11,...,1.133000e+09,107.94,99.51,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-10
9,1301,極洋,日本基準,連結,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2.266260e+11,...,1.799000e+09,171.33,157.94,2.306500e+10,9.460800e+10,2149.67,2.689000e+09,-5.114000e+09,2.482000e+09,2017-05-11
11,1301,極洋,日本基準,連結,2017年3月期,第1四半期,2016-04-01,2016-06-30,売上高,5.220600e+10,...,5.510000e+08,52.47,48.37,NaN,NaN,NaN,NaN,NaN,NaN,2017-08-04
12,1301,極洋,日本基準,連結,2017年3月期,第2四半期,2016-04-01,2016-09-30,売上高,1.095700e+11,...,1.186000e+09,112.99,104.16,NaN,NaN,NaN,NaN,NaN,NaN,2017-11-06


In [16]:
kessan_table_new_csv.describe()

,証券コード,売上高,営業利益,経常利益,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー
count,58969.000000,5.889000e+04,5.641200e+04,5.894100e+04,5.895800e+04,58928.000000,15753.000000,2.956500e+04,2.956500e+04,21887.000000,9.614000e+03,9.611000e+03,9.577000e+03
mean,5832.302922,1.237426e+11,7.602876e+09,9.447355e+09,6.089828e+09,49.314229,66.515108,1.241750e+11,6.906529e+11,1119.091200,2.651936e+10,-2.702570e+09,-3.231654e+09
std,2428.888901,5.830953e+11,4.628122e+10,5.723010e+10,4.223220e+10,108.580784,95.355665,6.448516e+11,8.132317e+12,1339.878413,2.537095e+11,3.311693e+11,6.130280e+10
min,1301.000000,-3.720000e+08,-5.762770e+11,-5.970170e+11,-1.092006e+12,-4468.650000,-185.370000,-7.303370e+11,1.060000e+08,-3308.020000,-2.922978e+12,-4.213597e+12,-2.130644e+12
25%,3676.000000,4.971000e+09,1.350000e+08,1.580000e+08,8.600000e+07,6.270000,15.840000,4.652000e+09,9.882000e+09,346.305000,3.450000e+08,-3.843000e+09,-1.866000e+09
50%,6113.000000,1.663200e+10,6.890000e+08,8.040000e+08,5.040000e+08,27.810000,38.080000,1.556300e+10,3.091600e+10,750.370000,1.568500e+09,-7.510000e+08,-3.430000e+08
75%,7912.000000,5.783225e+10,2.974000e+09,3.609000e+09,2.354000e+09,69.752500,81.680000,5.644600e+10,1.115610e+11,1419.360000,6.863750e+09,-1.090000e+08,9.700000e+07
max,9997.000000,2.840312e+13,2.853971e+12,2.983381e+12,2.745949e+12,2903.850000,2838.970000,1.940619e+13,3.042216e+14,21438.660000,9.752429e+12,1.552178e+13,2.380746e+12


## 整形

In [52]:
# 日付のパース、datetime.dateへの型変換
kessan_table_new_csv['期首'] = kessan_table_new_csv['期首'].apply(lambda x: parse(x).date())
kessan_table_new_csv['期末'] = kessan_table_new_csv['期末'].apply(lambda x: parse(x).date())
kessan_table_new_csv['情報公開又は更新日'] = kessan_table_new_csv['情報公開又は更新日'].apply(lambda x: parse(x).date())

In [ ]:
# 同じく日付のパース、datetime.dateへの型変換をまとめて。1列ずつやるより少し遅い？ 23s -> 30s
kessan_table_new_csv[['期首', '期末', '情報公開又は更新日']] = kessan_table_new_csv[['期首', '期末', '情報公開又は更新日']].applymap(lambda x: parse(x).date())

In [54]:
# pandasのTimestampへの型変換
# まとめてできないの？
# kessan_table_new_csv[['期首', '期末', '情報公開又は更新日']] = pd.to_datetime(kessan_table_new_csv[['期首', '期末', '情報公開又は更新日']], format='%Y-%m-%d')
kessan_table_new_csv['期首'] = pd.to_datetime(kessan_table_new_csv['期首'], format='%Y-%m-%d')
kessan_table_new_csv['期末'] = pd.to_datetime(kessan_table_new_csv['期末'], format='%Y-%m-%d')
kessan_table_new_csv['情報公開又は更新日'] = pd.to_datetime(kessan_table_new_csv['情報公開又は更新日'], format='%Y-%m-%d')

In [18]:
# 数値に変換する列のリスト作成
num_list = [
    '売上高', 
    '営業利益', 
    '経常利益', 
    '純利益', 
    '一株当り純利益',
    '希薄化後一株当り純利益', 
    '純資産又は株主資本', 
    '総資産', 
    '一株当り純資産', 
    '営業キャッシュフロー',
    '投資キャッシュフロー', 
    '財務キャッシュフロー'
]

In [55]:
# 数値に変換する項目の「,」を削除、float型に変換
kessan_table_new_csv[num_list] = kessan_table_new_csv[num_list].apply(lambda x: x.str.replace(',','')).astype(float)

In [ ]:
# 冗長なやり方

# 2つのリストから1つの辞書を作成する方法
# num_dict = dict(zip(num_list, [str for i in range(len(num_list))]))

kessan_table_new_csv = kessan_table_new_csv.astype(dict(zip(num_list, [str for i in range(len(num_list))])))
kessan_table_new_csv[num_list] = kessan_table_new_csv[num_list].applymap(lambda x: x.replace(',', ''))
kessan_table_new_csv = kessan_table_new_csv.astype(dict(zip(num_list, [float for i in range(len(num_list))])))

In [ ]:
# さらに冗長なやり方

kessan_table_new_csv = kessan_table_new_csv.astype({
    '売上高': str, 
    '営業利益': str, 
    '経常利益': str, 
    '純利益': str, 
    '一株当り純利益': str,
    '希薄化後一株当り純利益': str, 
    '純資産又は株主資本': str, 
    '総資産': str, 
    '一株当り純資産': str, 
    '営業キャッシュフロー': str,
    '投資キャッシュフロー': str, 
    '財務キャッシュフロー': str})

kessan_table_new_csv[[
    '売上高', 
    '営業利益', 
    '経常利益', 
    '純利益', 
    '一株当り純利益',
    '希薄化後一株当り純利益', 
    '純資産又は株主資本', 
    '総資産', 
    '一株当り純資産', 
    '営業キャッシュフロー',
    '投資キャッシュフロー', 
    '財務キャッシュフロー'
    ]] = kessan_table_new_csv[[
        '売上高', 
        '営業利益', 
        '経常利益', 
        '純利益', 
        '一株当り純利益',
        '希薄化後一株当り純利益', 
        '純資産又は株主資本', 
        '総資産', 
        '一株当り純資産', 
        '営業キャッシュフロー',
        '投資キャッシュフロー', 
        '財務キャッシュフロー'
        ]].applymap(lambda x: x.replace(',', ''))

kessan_table_new_csv = kessan_table_new_csv.astype({
    '売上高': float, 
    '営業利益': float, 
    '経常利益': float, 
    '純利益': float, 
    '一株当り純利益': float,
    '希薄化後一株当り純利益': float, 
    '純資産又は株主資本': float, 
    '総資産': float, 
    '一株当り純資産': float, 
    '営業キャッシュフロー': float,
       '投資キャッシュフロー': float, 
    '財務キャッシュフロー': float})

In [57]:
# ソート
kessan_table_new_csv = kessan_table_new_csv.sort_values(['証券コード', '連結個別', '期末', '情報公開又は更新日']).reset_index(drop=True)

# 20150521f の読み込み、整形、保存 

## 20150521fをxlsのまま読み込む  

In [248]:
# 決算プロの決算短信xlsを読み込む
kessan_table_15 = pd.read_excel('/Users/Really/Stockyard/_dl_data/20150521f.xls') # http://ke.kabupro.jp/doc/down40.htm

## 内容の確認

In [29]:
kessan_table_15

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目（売上高欄に掲載）,売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開又は更新日
0,1301,極洋,日本基準,連結,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.183500e+11,...,2433000000,23.17,21.36,2.306900e+10,8.893700e+10,215.65,-2.340000e+09,-7.620000e+08,3.698000e+09,2015-05-08
1,1301,極洋,日本基準,連結,2015年3月期,第3四半期,2014-04-01,2014-12-31,売上高,1.701400e+11,...,2806000000,26.72,24.63,2.360300e+10,1.068710e+11,NaN,NaN,NaN,NaN,2015-02-04
2,1301,極洋,日本基準,連結,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,1.024760e+11,...,2265000000,21.57,19.89,2.233800e+10,9.572100e+10,NaN,NaN,NaN,NaN,2014-11-07
3,1301,極洋,日本基準,連結,2015年3月期,第1四半期,2014-04-01,2014-06-30,売上高,4.867500e+10,...,223000000,2.13,1.96,1.917100e+10,9.166900e+10,NaN,NaN,NaN,NaN,2014-08-05
4,1301,極洋,日本基準,連結,2014年3月期,通期,2013-04-01,2014-03-31,売上高,2.023870e+11,...,2968000000,28.26,27.55,1.993000e+10,8.431900e+10,187.57,1.910000e+09,-1.900000e+09,-5.120000e+08,2015-05-08
5,1301,極洋,日本基準,連結,2014年3月期,第3四半期,2013-04-01,2013-12-31,売上高,1.571560e+11,...,2724000000,25.94,25.77,NaN,NaN,NaN,NaN,NaN,NaN,2015-02-04
6,1301,極洋,日本基準,連結,2014年3月期,第2四半期,2013-04-01,2013-09-30,売上高,9.410100e+10,...,1880000000,17.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-11-07
7,1301,極洋,日本基準,連結,2014年3月期,第1四半期,2013-04-01,2013-06-30,売上高,4.485800e+10,...,877000000,8.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-08-05
8,1301,極洋,日本基準,連結,2013年3月期,通期,2012-04-01,2013-03-31,売上高,1.780460e+11,...,1269000000,12.08,NaN,1.868300e+10,8.324500e+10,175.54,7.384000e+09,-4.690000e+08,-6.614000e+09,2014-05-09
9,1301,極洋,日本基準,連結,2013年3月期,第3四半期,2012-04-01,2012-12-31,売上高,1.377930e+11,...,952000000,9.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-02-05


In [78]:
kessan_table_15[['証券コード', '企業名', '会計基準', '連結個別', '決算期', '決算期間', '期首', '期末',
       '名寄前勘定科目（売上高欄に掲載）', '情報公開又は更新日']]

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目（売上高欄に掲載）,情報公開又は更新日
0,1301,極洋,日本基準,個別,2013年3月期,第2四半期,2012-04-01,2012-09-30,売上高,2013-10-21
1,1301,極洋,日本基準,個別,2013年3月期,通期,2012-04-01,2013-03-31,売上高,2014-05-09
2,1301,極洋,日本基準,個別,2014年3月期,第2四半期,2013-04-01,2013-09-30,売上高,2014-10-27
3,1301,極洋,日本基準,個別,2014年3月期,通期,2013-04-01,2014-03-31,売上高,2015-05-08
4,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2015-05-08
5,1301,極洋,日本基準,連結,2013年3月期,第1四半期,2012-04-01,2012-06-30,売上高,2013-08-05
6,1301,極洋,日本基準,連結,2013年3月期,第2四半期,2012-04-01,2012-09-30,売上高,2013-10-21
7,1301,極洋,日本基準,連結,2013年3月期,第3四半期,2012-04-01,2012-12-31,売上高,2014-02-05
8,1301,極洋,日本基準,連結,2013年3月期,通期,2012-04-01,2013-03-31,売上高,2014-05-09
9,1301,極洋,日本基準,連結,2014年3月期,第1四半期,2013-04-01,2013-06-30,売上高,2014-08-05


In [32]:
kessan_table_15[['証券コード', '企業名', '期末', '売上高', '営業利益', '経常利益', '純利益', '情報公開又は更新日']]

,証券コード,企業名,期末,売上高,営業利益,経常利益,純利益,情報公開又は更新日
0,1301,極洋,2015-03-31,2.183500e+11,2.460000e+09,2.107000e+09,2433000000,2015-05-08
1,1301,極洋,2014-12-31,1.701400e+11,2.293000e+09,1.974000e+09,2806000000,2015-02-04
2,1301,極洋,2014-09-30,1.024760e+11,7.210000e+08,6.860000e+08,2265000000,2014-11-07
3,1301,極洋,2014-06-30,4.867500e+10,3.190000e+08,3.890000e+08,223000000,2014-08-05
4,1301,極洋,2014-03-31,2.023870e+11,2.915000e+09,2.985000e+09,2968000000,2015-05-08
5,1301,極洋,2013-12-31,1.571560e+11,2.875000e+09,2.950000e+09,2724000000,2015-02-04
6,1301,極洋,2013-09-30,9.410100e+10,1.404000e+09,1.500000e+09,1880000000,2014-11-07
7,1301,極洋,2013-06-30,4.485800e+10,1.006000e+09,1.104000e+09,877000000,2014-08-05
8,1301,極洋,2013-03-31,1.780460e+11,2.324000e+09,2.262000e+09,1269000000,2014-05-09
9,1301,極洋,2012-12-31,1.377930e+11,1.824000e+09,1.960000e+09,952000000,2014-02-05


In [33]:
kessan_table_15[['証券コード', '企業名', '期末', '一株当り純利益', '希薄化後一株当り純利益', '純資産又は株主資本',
       '総資産', '一株当り純資産', '営業キャッシュフロー', '投資キャッシュフロー', '財務キャッシュフロー',
       '情報公開又は更新日']]

,証券コード,企業名,期末,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開又は更新日
0,1301,極洋,2015-03-31,23.17,21.36,2.306900e+10,8.893700e+10,215.65,-2.340000e+09,-7.620000e+08,3.698000e+09,2015-05-08
1,1301,極洋,2014-12-31,26.72,24.63,2.360300e+10,1.068710e+11,NaN,NaN,NaN,NaN,2015-02-04
2,1301,極洋,2014-09-30,21.57,19.89,2.233800e+10,9.572100e+10,NaN,NaN,NaN,NaN,2014-11-07
3,1301,極洋,2014-06-30,2.13,1.96,1.917100e+10,9.166900e+10,NaN,NaN,NaN,NaN,2014-08-05
4,1301,極洋,2014-03-31,28.26,27.55,1.993000e+10,8.431900e+10,187.57,1.910000e+09,-1.900000e+09,-5.120000e+08,2015-05-08
5,1301,極洋,2013-12-31,25.94,25.77,NaN,NaN,NaN,NaN,NaN,NaN,2015-02-04
6,1301,極洋,2013-09-30,17.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-11-07
7,1301,極洋,2013-06-30,8.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-08-05
8,1301,極洋,2013-03-31,12.08,NaN,1.868300e+10,8.324500e+10,175.54,7.384000e+09,-4.690000e+08,-6.614000e+09,2014-05-09
9,1301,極洋,2012-12-31,9.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-02-05


In [303]:
kessan_table_15.dtypes

証券コード                        int64
企業名                         object
会計基準                        object
連結個別                        object
決算期                         object
決算期間                        object
期首                  datetime64[ns]
期末                  datetime64[ns]
名寄前勘定科目（売上高欄に掲載）            object
売上高                        float64
営業利益                       float64
経常利益                       float64
純利益                          int64
一株当り純利益                    float64
希薄化後一株当り純利益                 object
純資産又は株主資本                  float64
総資産                        float64
一株当り純資産                    float64
営業キャッシュフロー                 float64
投資キャッシュフロー                 float64
財務キャッシュフロー                 float64
情報公開又は更新日           datetime64[ns]
dtype: object

In [4]:
# 2015年のファイルでは int
kessan_table_15['純利益'].max()

2173338000000

In [35]:
# 2017年のファイルでは float
kessan_table_new['純利益'].max()

2745949000000.0

In [6]:
# 2015年のファイルでは文字列が混入しているため object
kessan_table_15['希薄化後一株当り純利益'].max()

TypeError: '>=' not supported between instances of 'float' and 'str'

In [7]:
kessan_table_15['希薄化後一株当り純利益'].min()

TypeError: '<=' not supported between instances of 'float' and 'str'

In [75]:
kessan_table_15['期末'].min()

Timestamp('2012-02-20 00:00:00')

In [76]:
kessan_table_15['情報公開又は更新日'].min()

Timestamp('2012-04-26 00:00:00')

In [73]:
kessan_table_15.duplicated().any()

False

In [41]:
kessan_table_15.isnull().any()

証券コード               False
企業名                 False
会計基準                False
連結個別                False
決算期                 False
決算期間                False
期首                  False
期末                  False
名寄前勘定科目（売上高欄に掲載）     True
売上高                  True
営業利益                 True
経常利益                 True
純利益                 False
一株当り純利益              True
希薄化後一株当り純利益          True
純資産又は株主資本            True
総資産                  True
一株当り純資産              True
営業キャッシュフロー           True
投資キャッシュフロー           True
財務キャッシュフロー           True
情報公開又は更新日           False
dtype: bool

## 整形

In [304]:
# int型だった
kessan_table_15['純利益'] = kessan_table_15['純利益'].astype(float)

In [249]:
# index 17979 に '―' が入っていたために列が object 型になってしまっている
kessan_table_15[kessan_table_15['希薄化後一株当り純利益'].apply(lambda x: type(x) is str)]

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目（売上高欄に掲載）,売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開又は更新日
17979,4619,日本特殊塗料,日本基準,連結,2013年3月期,第3四半期,2012-04-01,2012-12-31,売上高,2.477800e+10,...,1181000000,53.44,―,NaN,NaN,NaN,NaN,NaN,NaN,2014-02-12


In [305]:
kessan_table_15.loc[kessan_table_15['希薄化後一株当り純利益'].apply(lambda x: type(x) is str), '希薄化後一株当り純利益'] = np.nan

In [218]:
kessan_table_15[kessan_table_15['希薄化後一株当り純利益'].apply(lambda x: type(x) is str)]

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目（売上高欄に掲載）,売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開又は更新日


In [219]:
kessan_table_15['希薄化後一株当り純利益'][17979]

nan

In [223]:
kessan_table_15['希薄化後一株当り純利益'].max()

15323.77

In [306]:
kessan_table_15['希薄化後一株当り純利益'] = kessan_table_15['希薄化後一株当り純利益'].astype(float)

In [307]:
# ソート
kessan_table_15 = kessan_table_15.sort_values(['証券コード', '連結個別', '期末', '情報公開又は更新日']).reset_index(drop=True)

## 保存

In [79]:
# 保存
kessan_table_15.to_csv('/Users/Really/Stockyard/_csv/kessan_table_20150521.csv', index=False)

In [128]:
# 保存したファイルの確認
df = pd.read_csv('/Users/Really/Stockyard/_csv/kessan_table_20150521.csv')
df

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目（売上高欄に掲載）,売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開又は更新日
0,1301,極洋,日本基準,個別,2013年3月期,第2四半期,2012-04-01,2012-09-30,売上高,7.654600e+10,...,7.900000e+07,0.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-10-21
1,1301,極洋,日本基準,個別,2013年3月期,通期,2012-04-01,2013-03-31,売上高,1.606100e+11,...,8.060000e+08,7.68,NaN,1.481800e+10,7.049200e+10,141.08,NaN,NaN,NaN,2014-05-09
2,1301,極洋,日本基準,個別,2014年3月期,第2四半期,2013-04-01,2013-09-30,売上高,8.816100e+10,...,1.834000e+09,17.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-10-27
3,1301,極洋,日本基準,個別,2014年3月期,通期,2013-04-01,2014-03-31,売上高,1.920260e+11,...,1.806000e+09,17.20,16.77,1.616400e+10,7.038100e+10,153.90,NaN,NaN,NaN,2015-05-08
4,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.120860e+11,...,2.139000e+09,20.37,18.78,1.769500e+10,7.351900e+10,168.48,NaN,NaN,NaN,2015-05-08
5,1301,極洋,日本基準,連結,2013年3月期,第1四半期,2012-04-01,2012-06-30,売上高,4.319100e+10,...,3.350000e+08,3.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-05
6,1301,極洋,日本基準,連結,2013年3月期,第2四半期,2012-04-01,2012-09-30,売上高,8.554100e+10,...,1.600000e+08,1.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-10-21
7,1301,極洋,日本基準,連結,2013年3月期,第3四半期,2012-04-01,2012-12-31,売上高,1.377930e+11,...,9.520000e+08,9.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-02-05
8,1301,極洋,日本基準,連結,2013年3月期,通期,2012-04-01,2013-03-31,売上高,1.780460e+11,...,1.269000e+09,12.08,NaN,1.868300e+10,8.324500e+10,175.54,7.384000e+09,-4.690000e+08,-6.614000e+09,2014-05-09
9,1301,極洋,日本基準,連結,2014年3月期,第1四半期,2013-04-01,2013-06-30,売上高,4.485800e+10,...,8.770000e+08,8.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-08-05


# 新旧ファイルの結合

In [82]:
kessan_table = kessan_table_new.append(kessan_table_15)

In [95]:
kessan_table

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目（売上高欄に掲載）,売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開又は更新日
0,1301,極洋,日本基準,個別,2013年3月期,第2四半期,2012-04-01,2012-09-30,売上高,7.654600e+10,...,7.900000e+07,0.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-10-21
1,1301,極洋,日本基準,個別,2013年3月期,通期,2012-04-01,2013-03-31,売上高,1.606100e+11,...,8.060000e+08,7.68,NaN,1.481800e+10,7.049200e+10,141.08,NaN,NaN,NaN,2014-05-09
2,1301,極洋,日本基準,個別,2014年3月期,第2四半期,2013-04-01,2013-09-30,売上高,8.816100e+10,...,1.834000e+09,17.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-10-27
3,1301,極洋,日本基準,個別,2014年3月期,通期,2013-04-01,2014-03-31,売上高,1.920260e+11,...,1.806000e+09,17.20,16.77,1.616400e+10,7.038100e+10,153.90,NaN,NaN,NaN,2015-05-08
4,1301,極洋,日本基準,個別,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,9.947600e+10,...,2.504000e+09,23.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-26
5,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.120860e+11,...,2.139000e+09,20.37,18.78,1.769500e+10,7.351900e+10,168.48,NaN,NaN,NaN,2015-05-08
6,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.120860e+11,...,2.139000e+09,20.37,18.78,1.769500e+10,7.351900e+10,168.48,NaN,NaN,NaN,2016-05-09
7,1301,極洋,日本基準,個別,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2.163500e+11,...,1.259000e+09,119.87,110.51,1.781700e+10,7.814000e+10,1696.42,NaN,NaN,NaN,2017-05-11
8,1301,極洋,日本基準,個別,2017年3月期,通期,2016-04-01,2017-03-31,売上高,2.280830e+11,...,2.194000e+09,208.94,192.95,2.011700e+10,8.130600e+10,1915.42,NaN,NaN,NaN,2017-05-11
9,1301,極洋,日本基準,連結,2013年3月期,第1四半期,2012-04-01,2012-06-30,売上高,4.319100e+10,...,3.350000e+08,3.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-05


In [84]:
# ソート
kessan_table = kessan_table.sort_values(['証券コード', '連結個別', '期末', '情報公開又は更新日']).reset_index(drop=True)

In [86]:
kessan_table.dtypes

証券コード                        int64
企業名                         object
会計基準                        object
連結個別                        object
決算期                         object
決算期間                        object
期首                  datetime64[ns]
期末                  datetime64[ns]
名寄前勘定科目（売上高欄に掲載）            object
売上高                        float64
営業利益                       float64
経常利益                       float64
純利益                        float64
一株当り純利益                    float64
希薄化後一株当り純利益                float64
純資産又は株主資本                  float64
総資産                        float64
一株当り純資産                    float64
営業キャッシュフロー                 float64
投資キャッシュフロー                 float64
財務キャッシュフロー                 float64
情報公開又は更新日           datetime64[ns]
dtype: object

## 重複の確認、削除

In [97]:
kessan_table.duplicated().any()

False

In [1]:
# keep=Falseで重複行のいずれかではなくすべてを表示
kessan_table[kessan_table.duplicated(keep=False)]

NameError: name 'kessan_table' is not defined

In [92]:
kessan_table = kessan_table.drop_duplicates()

## 内容の確認

In [96]:
kessan_table.isnull().any()

証券コード               False
企業名                 False
会計基準                False
連結個別                False
決算期                 False
決算期間                False
期首                  False
期末                  False
名寄前勘定科目（売上高欄に掲載）     True
売上高                  True
営業利益                 True
経常利益                 True
純利益                  True
一株当り純利益              True
希薄化後一株当り純利益          True
純資産又は株主資本            True
総資産                  True
一株当り純資産              True
営業キャッシュフロー           True
投資キャッシュフロー           True
財務キャッシュフロー           True
情報公開又は更新日           False
dtype: bool

In [98]:
kessan_table['期末'].min()

Timestamp('2012-02-20 00:00:00')

In [99]:
kessan_table['情報公開又は更新日'].min()

Timestamp('2012-04-26 00:00:00')

In [100]:
kessan_table['期末'].max()

Timestamp('2017-09-30 00:00:00')

In [101]:
kessan_table['情報公開又は更新日'].max()

Timestamp('2017-11-10 00:00:00')

In [38]:
kessan_table.describe()

,証券コード,売上高,営業利益,経常利益,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー
count,58969.000000,5.889000e+04,5.641200e+04,5.894100e+04,5.895800e+04,58928.000000,15753.000000,2.956500e+04,2.956500e+04,21887.000000,9.614000e+03,9.611000e+03,9.577000e+03
mean,5832.302922,1.237426e+11,7.602876e+09,9.447355e+09,6.089828e+09,49.314229,66.515108,1.241750e+11,6.906529e+11,1119.091200,2.651936e+10,-2.702570e+09,-3.231654e+09
std,2428.888901,5.830953e+11,4.628122e+10,5.723010e+10,4.223220e+10,108.580784,95.355665,6.448516e+11,8.132317e+12,1339.878413,2.537095e+11,3.311693e+11,6.130280e+10
min,1301.000000,-3.720000e+08,-5.762770e+11,-5.970170e+11,-1.092006e+12,-4468.650000,-185.370000,-7.303370e+11,1.060000e+08,-3308.020000,-2.922978e+12,-4.213597e+12,-2.130644e+12
25%,3676.000000,4.971000e+09,1.350000e+08,1.580000e+08,8.600000e+07,6.270000,15.840000,4.652000e+09,9.882000e+09,346.305000,3.450000e+08,-3.843000e+09,-1.866000e+09
50%,6113.000000,1.663200e+10,6.890000e+08,8.040000e+08,5.040000e+08,27.810000,38.080000,1.556300e+10,3.091600e+10,750.370000,1.568500e+09,-7.510000e+08,-3.430000e+08
75%,7912.000000,5.783225e+10,2.974000e+09,3.609000e+09,2.354000e+09,69.752500,81.680000,5.644600e+10,1.115610e+11,1419.360000,6.863750e+09,-1.090000e+08,9.700000e+07
max,9997.000000,2.840312e+13,2.853971e+12,2.983381e+12,2.745949e+12,2903.850000,2838.970000,1.940619e+13,3.042216e+14,21438.660000,9.752429e+12,1.552178e+13,2.380746e+12


In [102]:
# 保存
kessan_table.to_csv('/Users/Really/Stockyard/_csv/kessan_table.csv', index=False)

In [80]:
# 保存したファイルの確認
df = pd.read_csv('/Users/Really/Stockyard/_csv/kessan_table.csv')
df

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目（売上高欄に掲載）,売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開又は更新日
0,1301,極洋,日本基準,個別,2013年3月期,第2四半期,2012-04-01,2012-09-30,売上高,7.654600e+10,...,7.900000e+07,0.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-10-21
1,1301,極洋,日本基準,個別,2013年3月期,通期,2012-04-01,2013-03-31,売上高,1.606100e+11,...,8.060000e+08,7.68,NaN,1.481800e+10,7.049200e+10,141.08,NaN,NaN,NaN,2014-05-09
2,1301,極洋,日本基準,個別,2014年3月期,第2四半期,2013-04-01,2013-09-30,売上高,8.816100e+10,...,1.834000e+09,17.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-10-27
3,1301,極洋,日本基準,個別,2014年3月期,通期,2013-04-01,2014-03-31,売上高,1.920260e+11,...,1.806000e+09,17.20,16.77,1.616400e+10,7.038100e+10,153.90,NaN,NaN,NaN,2015-05-08
4,1301,極洋,日本基準,個別,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,9.947600e+10,...,2.504000e+09,23.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-26
5,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.120860e+11,...,2.139000e+09,20.37,18.78,1.769500e+10,7.351900e+10,168.48,NaN,NaN,NaN,2015-05-08
6,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.120860e+11,...,2.139000e+09,20.37,18.78,1.769500e+10,7.351900e+10,168.48,NaN,NaN,NaN,2016-05-09
7,1301,極洋,日本基準,個別,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2.163500e+11,...,1.259000e+09,119.87,110.51,1.781700e+10,7.814000e+10,1696.42,NaN,NaN,NaN,2017-05-11
8,1301,極洋,日本基準,個別,2017年3月期,通期,2016-04-01,2017-03-31,売上高,2.280830e+11,...,2.194000e+09,208.94,192.95,2.011700e+10,8.130600e+10,1915.42,NaN,NaN,NaN,2017-05-11
9,1301,極洋,日本基準,連結,2013年3月期,第1四半期,2012-04-01,2012-06-30,売上高,4.319100e+10,...,3.350000e+08,3.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-05
